# Adding widgets

The bokeh plotting feature already gives us some interaction possibilities, but we can customize with our own widgets

In [ ]:
import pandas as pd
import numpy as np
from stats_tennis import player_stats
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

In [ ]:
output_notebook()

In [ ]:
matches = pd.read_csv('wta_matches_2018.csv', parse_dates=True)
stats = player_stats(matches)

In [ ]:
source = ColumnDataSource(stats)

## The Select widget

A great thing to have on your dashboard is a dropdown menu where you can choose between some options

In [ ]:
from bokeh.models.widgets import Select

In [ ]:
show(row(Select()))

In [ ]:
tennis_stuff = ['Racket', 'Shoes', 'Tennis ball']
select_tennis_stuff = Select(value='Racket', options=tennis_stuff)

In [ ]:
show(row(select_tennis_stuff))

In [ ]:
list(stats.columns)

In [ ]:
select_column = Select(value=list(stats.columns)[0], options=list(stats.columns))

In [ ]:
show(row(select_column))

Now we want to be able to change the X and Y axis to waht we want to look at.

In [ ]:
# Our code so far

select_column_x1 = Select(value='Rank', options=list(stats.columns))
select_column_y1 = Select(value='Wins', options=list(stats.columns))
select_column_x2 = Select(value='Rank', options=list(stats.columns))
select_column_y2 = Select(value='Losses', options=list(stats.columns))

fig1 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset')
circles1 = fig1.circle(x='Rank', y='Wins', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
fig2 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset')
circles2 = fig2.circle(x='Rank', y='Losses', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)

show(row(column(select_column_x1,
                select_column_y1,
                fig1),
         column(select_column_x2,
                select_column_y2,
                fig2)))

We need to do two things:

1. The x and y values in the plot's cannot be set as they are now with 'Rank', 'Wins' and 'Losses'.
2. We need to change these variables when a new item on the dropdown menu has been chosen

In [ ]:
# 1

source.data = dict(x1=stats[select_column_x1.value],
                   y1=stats[select_column_y1.value],
                   x2=stats[select_column_x2.value],
                   y2=stats[select_column_y2.value])

In [ ]:
fig1 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset')
circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
fig2 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset')
circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)

show(row(column(select_column_x1,
                select_column_y1,
                fig1),
         column(select_column_x2,
                select_column_y2,
                fig2)))

In [ ]:
selectors = [select_column_x1, select_column_y1,
             select_column_x2, select_column_y2]

def update_plots(attrname, old, new):
    """
    Update the plots in our bokeh dashboard
    """
    
    source.data = dict(x1=stats[select_column_x1.value],
                       y1=stats[select_column_y1.value],
                       x2=stats[select_column_x2.value],
                       y2=stats[select_column_y2.value])
        
for widget in selectors:
    widget.on_change('value', update_plots)

In [ ]:
show(row(column(select_column_x1,
                select_column_y1,
                fig1),
         column(select_column_x2,
                select_column_y2,
                fig2)))

We need to run a bokeh server!

In [ ]:
def modify_doc(doc):
    
    # Widgets
    
    select_column_x1 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y1 = Select(value='Wins', options=list(stats.columns[1:]))
    select_column_x2 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y2 = Select(value='Losses', options=list(stats.columns[1:]))
    
    selectors = [select_column_x1, select_column_y1,
                 select_column_x2, select_column_y2]
    
    # Source
    
    source = ColumnDataSource(data=dict(x1=stats[select_column_x1.value],
                                        y1=stats[select_column_y1.value],
                                        x2=stats[select_column_x2.value],
                                        y2=stats[select_column_y2.value]))
    
    # Plots
    
    fig1 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    fig2 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    
    
    # Callback
    
    def update_plots(attrname, old, new):
        """
        Update the plots in our bokeh dashboard
        """
        
        source.data = dict(x1=stats[select_column_x1.value],
                           y1=stats[select_column_y1.value],
                           x2=stats[select_column_x2.value],
                           y2=stats[select_column_y2.value])

    for widget in selectors:
        widget.on_change('value', update_plots)
    
    # Layout
    
    layout = row(column(select_column_x1,
                        select_column_y1,
                        fig1),
                 column(select_column_x2,
                        select_column_y2,
                        fig2))

    doc.add_root(layout)
    
show(modify_doc)

## Slider

We also want to include a slider. Let's be able to change the size of our points!

In [ ]:
from bokeh.models.widgets import Slider

In [ ]:
size_slider = Slider(start=1, end=20, value=3, title="Marker size")

In [ ]:
show(row(size_slider))

In [ ]:
def modify_doc(doc):
    
    # Widgets
    
    select_column_x1 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y1 = Select(value='Wins', options=list(stats.columns[1:]))
    select_column_x2 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y2 = Select(value='Losses', options=list(stats.columns[1:]))
    
    selectors = [select_column_x1, select_column_y1,
                 select_column_x2, select_column_y2]
    
    size_slider = Slider(start=1, end=20, value=3, title="Marker size")
    
    sliders = [size_slider]
    
    widgets = selectors + sliders
    
    # Source
    
    source = ColumnDataSource(data=dict(x1=stats[select_column_x1.value],
                                        y1=stats[select_column_y1.value],
                                        x2=stats[select_column_x2.value],
                                        y2=stats[select_column_y2.value]))
    
    # Plots
    
    fig1 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    fig2 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    
    
    # Callback
    
    def update_plots(attrname, old, new):
        """
        Update the plots in our bokeh dashboard
        """
        
        source.data = dict(x1=stats[select_column_x1.value],
                           y1=stats[select_column_y1.value],
                           x2=stats[select_column_x2.value],
                           y2=stats[select_column_y2.value])
            
        for circles in [circles1, circles2]:
            circles.glyph.size = size_slider.value

    for widget in widgets:
        widget.on_change('value', update_plots)
    
    # Layout
    
    layout = row(column(select_column_x1,
                        select_column_y1,
                        fig1, size_slider),
                 column(select_column_x2,
                        select_column_y2,
                        fig2))

    doc.add_root(layout)
    
show(modify_doc)

 We also want to limit the amount of players we look at by dragging a slider. For this we need a RangeSlider

In [ ]:
from bokeh.models.widgets import RangeSlider

In [ ]:
stats['Rank'].max()

Max rank is 1051 who played on the WTA tour last year.

In [ ]:
rank_slider = RangeSlider(start=0, end=max(stats['Rank']),
                              value=(0, max(stats['Rank'])), title='Rank interval')

In [ ]:
show(row(rank_slider))

In [ ]:
rank_slider.value

In [ ]:
def modify_doc(doc):
    
    # Widgets
    
    select_column_x1 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y1 = Select(value='Wins', options=list(stats.columns[1:]))
    select_column_x2 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y2 = Select(value='Losses', options=list(stats.columns[1:]))
    
    selectors = [select_column_x1, select_column_y1,
                 select_column_x2, select_column_y2]
    
    size_slider = Slider(start=1, end=20, value=3, title="Marker size")
    
    rank_slider = RangeSlider(start=0, end=max(stats['Rank']),
                              value=(0, max(stats['Rank'])), title='Rank interval')
    
    sliders = [size_slider, rank_slider]
    
    widgets = selectors + sliders
    
    # Source
    
    source = ColumnDataSource(data=dict(x1=stats[select_column_x1.value],
                                        y1=stats[select_column_y1.value],
                                        x2=stats[select_column_x2.value],
                                        y2=stats[select_column_y2.value]))
    
    # Plots
    
    fig1 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    fig2 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset')
    circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    
    
    # Callback
    
    def update_plots(attrname, old, new):
        """
        Update the plots in our bokeh dashboard
        """
        
        rank_interval = (stats['Rank'] > rank_slider.value[0]) & (stats['Rank'] < rank_slider.value[1])
        
        source.data = dict(x1=stats[rank_interval][select_column_x1.value],
                           y1=stats[rank_interval][select_column_y1.value],
                           x2=stats[rank_interval][select_column_x2.value],
                           y2=stats[rank_interval][select_column_y2.value])
            
        for circles in [circles1, circles2]:
            circles.glyph.size = size_slider.value

    for widget in widgets:
        widget.on_change('value', update_plots)
    
    # Layout
    
    layout = row(column(select_column_x1,
                        select_column_y1,
                        fig1, size_slider,
                        rank_slider),
                 column(select_column_x2,
                        select_column_y2,
                        fig2))

    doc.add_root(layout)
    
show(modify_doc)

# Tooltips

It could be nice to know which players we are looking at, so we can hover over them and get information.

Bokeh has nice documentation on this:

https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#userguide-tools-inspectors

In [ ]:
def modify_doc(doc):
    
    # Widgets
    
    select_column_x1 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y1 = Select(value='Wins', options=list(stats.columns[1:]))
    select_column_x2 = Select(value='Rank', options=list(stats.columns[1:]))
    select_column_y2 = Select(value='Losses', options=list(stats.columns[1:]))
    
    selectors = [select_column_x1, select_column_y1,
                 select_column_x2, select_column_y2]
    
    size_slider = Slider(start=1, end=20, value=3, title="Marker size")
    
    rank_slider = RangeSlider(start=0, end=max(stats['Rank']),
                              value=(0, max(stats['Rank'])), title='Rank interval')
    
    sliders = [size_slider, rank_slider]
    
    widgets = selectors + sliders
    
    # Tooltips
    
    TOOLTIPS = [
    ("", "@name")
    ]
    
    # Source
    
    source = ColumnDataSource(data=dict(x1=stats[select_column_x1.value],
                                        y1=stats[select_column_y1.value],
                                        x2=stats[select_column_x2.value],
                                        y2=stats[select_column_y2.value],
                                        name=stats['Name']))
    
    # Plots
    
    fig1 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset', tooltips=TOOLTIPS)
    circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    fig2 = figure(plot_width=350, plot_height=350,
                  tools='pan,wheel_zoom,box_select,box_zoom,reset', tooltips=TOOLTIPS)
    circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                           selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
    
    
    # Callback
    
    def update_plots(attrname, old, new):
        """
        Update the plots in our bokeh dashboard
        """
        
        rank_interval = (stats['Rank'] > rank_slider.value[0]) & (stats['Rank'] < rank_slider.value[1])
        
        source.data = dict(x1=stats[rank_interval][select_column_x1.value],
                           y1=stats[rank_interval][select_column_y1.value],
                           x2=stats[rank_interval][select_column_x2.value],
                           y2=stats[rank_interval][select_column_y2.value],
                           name=stats[rank_interval]['Name'])
            
        for circles in [circles1, circles2]:
            circles.glyph.size = size_slider.value

    for widget in widgets:
        widget.on_change('value', update_plots)
    
    # Layout
    
    layout = row(column(select_column_x1,
                        select_column_y1,
                        fig1, size_slider,
                        rank_slider),
                 column(select_column_x2,
                        select_column_y2,
                        fig2))

    doc.add_root(layout)
    
show(modify_doc)

# Bokeh server

We talked earlier about needing a bokeh server. Sometimes it is nice to have the dashboard seperate from a Jupyter Notebook. We need then to copy what we have in the modify_doc function and put it in a separate file. We also need to add all the package import.

In [ ]:
import pandas as pd
import numpy as np

from stats_tennis import player_stats
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, RangeSlider, Select
from bokeh.plotting import figure
from bokeh.io import curdoc, show

matches = pd.read_csv('wta_matches_2018.csv', parse_dates=True)
stats = player_stats(matches)

# Widgets
    
select_column_x1 = Select(value='Rank', options=list(stats.columns[1:]))
select_column_y1 = Select(value='Wins', options=list(stats.columns[1:]))
select_column_x2 = Select(value='Rank', options=list(stats.columns[1:]))
select_column_y2 = Select(value='Losses', options=list(stats.columns[1:]))

selectors = [select_column_x1, select_column_y1,
             select_column_x2, select_column_y2]

size_slider = Slider(start=1, end=20, value=3, title="Marker size")

rank_slider = RangeSlider(start=0, end=max(stats['Rank']),
                          value=(0, max(stats['Rank'])), title='Rank interval')

sliders = [size_slider, rank_slider]

widgets = selectors + sliders

# Tooltips

TOOLTIPS = [
("", "@name")
]

# Source

source = ColumnDataSource(data=dict(x1=stats[select_column_x1.value],
                                    y1=stats[select_column_y1.value],
                                    x2=stats[select_column_x2.value],
                                    y2=stats[select_column_y2.value],
                                    name=stats['Name']))

# Plots

fig1 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset', tooltips=TOOLTIPS)
circles1 = fig1.circle(x='x1', y='y1', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
fig2 = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,box_zoom,reset', tooltips=TOOLTIPS)
circles2 = fig2.circle(x='x2', y='y2', size=3, source=source,
                       selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)


# Callback

def update_plots(attrname, old, new):
    """
    Update the plots in our bokeh dashboard
    """

    rank_interval = (stats['Rank'] > rank_slider.value[0]) & (stats['Rank'] < rank_slider.value[1])

    source.data = dict(x1=stats[rank_interval][select_column_x1.value],
                       y1=stats[rank_interval][select_column_y1.value],
                       x2=stats[rank_interval][select_column_x2.value],
                       y2=stats[rank_interval][select_column_y2.value],
                       name=stats[rank_interval]['Name'])

    for circles in [circles1, circles2]:
        circles.glyph.size = size_slider.value

for widget in widgets:
    widget.on_change('value', update_plots)

# Layout

layout = row(column(select_column_x1,
                    select_column_y1,
                    fig1, size_slider,
                    rank_slider),
             column(select_column_x2,
                    select_column_y2,
                    fig2))

curdoc().add_root(layout)

# Try yourself

Choose whichever you want to do:

1. Change colors of the points
2. Change start axis values
3. Put on axis labels
4. Add a new slider
5. Add a new plot
6. Add a new feature